In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import json

In [ ]:
df2 = pd.read_json('data/data2/data2.json')

In [ ]:
#show first five rows of df
df2.head()

array(['Accelerometer', 'Barometer', 'Battery', 'Brightness', 'Gravity',
       'Gyroscope', 'Light', 'Location', 'Magnetometer', 'Metadata',
       'Orientation'], dtype=object)

In [ ]:
#what sensors are in the dataset?
df2['sensor'].unique()

array(['Accelerometer', 'Barometer', 'Battery', 'Brightness', 'Gravity',
       'Gyroscope', 'Light', 'Location', 'Magnetometer', 'Metadata',
       'Orientation'], dtype=object)

In [7]:
#show columns of df
df2.columns

Index(['sensor', 'time', 'seconds_elapsed', 'z', 'y', 'x', 'relativeAltitude',
       'pressure', 'batteryLevel', 'batteryState', 'lowPowerMode',
       'brightness', 'lux', 'bearingAccuracy', 'speedAccuracy',
       'verticalAccuracy', 'horizontalAccuracy', 'speed', 'bearing',
       'altitude', 'longitude', 'latitude', 'version', 'device name',
       'recording time', 'platform', 'appVersion', 'device id', 'sensors',
       'sampleRateMs', 'qz', 'qy', 'qx', 'qw', 'roll', 'pitch', 'yaw'],
      dtype='object')

In [12]:
grouped = df2.groupby(by='sensor')


In [19]:
#dröselt das json in einzelne CSV datein auf und speichert diese
for sensor, group in grouped:
    sensor_df = pd.DataFrame(group)
    # Speichert das DataFrame für den aktuellen Sensor in eine CSV-Datei
    sensor_df.to_csv(f'Data2_{sensor}_data.csv', index=False)

In [4]:
def df2_klassifizieren (
    df2: pd.DataFrame,
    sensor: str,
    schwellwert: float
) -> pd.DataFrame:
    """Klassifiziert die Daten des gegebenen Sensors in 'normal' und 'anomal'."""
    # Kopie des DataFrames erstellen
    df2_klassifiziert = df2.copy()
    # Spalte 'klassifikation' mit dem Wert 'normal' erstellen
    df2_klassifiziert['klassifikation'] = 'normal'
    # Alle Werte, die den Schwellwert überschreiten, als 'anomal' klassifizieren
    df2_klassifiziert.loc[df2_klassifiziert['value'] > schwellwert, 'klassifikation'] = 'anomal'
    # Klassifizierten DataFrame zurückgeben
    return df2_klassifiziert
